In [23]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from datasets import load_from_disk
from vllm import LLM, SamplingParams

In [6]:
test_dataset = load_from_disk('test_dataset')

In [7]:
model = 'Qwen/Qwen2-7B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model)

prompt_lst = []
label_lst = []

for prompt in test_dataset["messages"]:
    text = tokenizer.apply_chat_template(
        prompt, tokenize=False, add_generation_prompt=False
    )
    input = text.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    label = text.split('<|im_start|>assistant')[1]
    prompt_lst.append(input)
    label_lst.append(label)

In [10]:
peft_model_id = "qwen2-7b-rag-ko/checkpoint-141"
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=fine_tuned_model, tokenizer=tokenizer)
eos_token = tokenizer("<|im_end|>",add_special_tokens=False)["input_ids"][0]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCaus

In [11]:
def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    return outputs[0]['generated_text'][len(prompt):].strip()
 
for prompt, label in zip(prompt_lst[300:305], label_lst[300:305]):
    # print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:623: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Starting from v4.46, the `logits` model output will have the same type as the model (except at train 

    response:
롯데백화점의 '활기차고 재미있는 전통시장 만들기' 지원 대상은 서울 약수시장·방이시장, 인천 모래내시장, 부산 서면시장 등 8개 전통시장입니다[[ref4]].
    label:

롯데백화점의 '활기차고 재미있는 전통시장 만들기' 지원 대상이 되는 시장의 개수는 8개입니다. 지원 대상 시장으로는 서울 약수시장, 방이시장, 인천 모래내시장, 부산 서면시장 등이 포함됩니다 [[ref4]].<|im_end|>

--------------------------------------------------
    response:
박영선 의원이 외국인투자촉진법 개정에 반대하며 예산안 처리에 '빨간불'을 켜준 날짜는 2012년 12월 31일 오전입니다. 이날 민주당이 예산안 처리 조건으로 국가정보원 개혁안을 합의했지만, 박영선 의원 등 민주당 일부는 외국인투자촉진법 개정에 반대하여 예산안 처리를 저지했습니다 [[ref4]].
    label:

박영선 의원이 외국인투자촉진법(외촉법) 개정을 반대한 날짜는 12월 31일입니다. 이 날 박영선 의원은 예산안 처리 조건으로 내세운 국가정보원 개혁안에 합의한 후, 외촉법 개정에 반대하며 예산안 처리에 '빨간불'을 켰습니다. [[ref4]]<|im_end|>

--------------------------------------------------
    response:
이번 해에 국내 맥도날드 가맹점 수는 50개로 증가할 예정입니다. 한국 맥도날드는 올해 중 가맹점 50개를 새로 열어 총 103개의 가맹점을 운영할 계획이라고 발표했습니다. 이는 2010년부터 시작된 가맹사업을 통해 4년간 가맹점 53개를 열어온 후의 계획입니다 [[ref4]].
    label:

한국 맥도날드는 올해 중 가맹점 50개를 새로 열어 총 103개의 가맹점을 운영할 계획이라고 발표했습니다. 또한, 직영점도 50개를 새로 열어 올해 말까지 총 점포 수를 444개 정도로 늘릴 계획입니다[[ref4]].<|im_end|>



In [12]:
!wget https://raw.githubusercontent.com/ukairia777/LLM-Finetuning-tutorial/main/merge.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


--2025-01-08 05:56:12--  https://raw.githubusercontent.com/ukairia777/LLM-Finetuning-tutorial/main/merge.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1351 (1.3K) [text/plain]
Saving to: ‘merge.py.1’

merge.py.1          100%[===================>]   1.32K  --.-KB/s    in 0s      

2025-01-08 05:56:12 (99.0 MB/s) - ‘merge.py.1’ saved [1351/1351]



In [13]:
!python merge.py \
    --base_model_name_or_path Qwen/Qwen2-7B-Instruct \
    --peft_model_path ./qwen2-7b-rag-ko/checkpoint-141 \
    --output_dir ./output_dir

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading base model: Qwen/Qwen2-7B-Instruct
Loading checkpoint shards: 100%|██████████████████| 4/4 [00:28<00:00,  7.07s/it]
Loading PEFT: ./qwen2-7b-rag-ko/checkpoint-141
Running merge_and_unload
Model saved to ./output_dir


In [17]:
model_id = './output_dir'
model = AutoModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", torch_dtype=torch.float16)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

for prompt, label in zip(prompt_lst[300:305], label_lst[300:305]):
    # print(f"    prompt:\n{prompt}")
    print(f"    response:\n{test_inference(pipe, prompt)}")
    print(f"    label:\n{label}")
    print("-"*50)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

    response:
롯데백화점의 '활기차고 재미있는 전통시장 만들기' 지원 대상은 서울 약수시장·방이시장, 인천 모래내시장, 부산 서면시장 등 8개 전통시장입니다[[ref4]].
    label:

롯데백화점의 '활기차고 재미있는 전통시장 만들기' 지원 대상이 되는 시장의 개수는 8개입니다. 지원 대상 시장으로는 서울 약수시장, 방이시장, 인천 모래내시장, 부산 서면시장 등이 포함됩니다 [[ref4]].<|im_end|>

--------------------------------------------------
    response:
박영선 의원이 외국인투자촉진법 개정을 반대한 날짜는 2012년 12월 31일 오전입니다. 박영선 의원은 외국인투자촉진법 개정을 처리하지 않으면 국가정보원 개혁안을 처리하지 않겠다고 주장했으나, 결국 외국인투자촉진법 개정을 처리하지 않으면 국가정보원 개혁안을 처리하지 않겠다는 새누리당의 입장에 맞서 외국인투자촉진법 개정을 반대했습니다 [[ref4]].
    label:

박영선 의원이 외국인투자촉진법(외촉법) 개정을 반대한 날짜는 12월 31일입니다. 이 날 박영선 의원은 예산안 처리 조건으로 내세운 국가정보원 개혁안에 합의한 후, 외촉법 개정에 반대하며 예산안 처리에 '빨간불'을 켰습니다. [[ref4]]<|im_end|>

--------------------------------------------------
    response:
이번 해에 국내 맥도날드 가맹점 수는 50개로 증가할 예정입니다. 한국 맥도날드는 올해 중 가맹점 50개를 새로 열어 총 103개의 가맹점을 운영할 계획이라고 발표했습니다. 이는 2010년부터 시작된 가맹사업을 통해 4년간 가맹점 53개를 열어온 후의 계획입니다 [[ref4]].
    label:

한국 맥도날드는 올해 중 가맹점 50개를 새로 열어 총 103개의 가맹점을 운영할 계획이라고 발표했습니다. 또한, 직영점도 50개를 새로 열어 올해 말까지 총 점포 수를 444개 정도로

In [18]:
from huggingface_hub import HfApi
api = HfApi()

username = "iamjoon"

In [19]:
MODEL_NAME = 'qwen2-7b-rag-ko-checkpoint-141'

In [22]:
api.create_repo(
    token="hf_여러분의 KEY",
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)

api.upload_folder(
    token="hf_여러분의 KEY",
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="output_dir",
)

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/iamjoon/qwen2-7b-rag-ko-checkpoint-141/commit/ca0e47a492ac9cfad8411de952460cf2aaeeedc7', commit_message='Upload folder using huggingface_hub', commit_description='', oid='ca0e47a492ac9cfad8411de952460cf2aaeeedc7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/iamjoon/qwen2-7b-rag-ko-checkpoint-141', endpoint='https://huggingface.co', repo_type='model', repo_id='iamjoon/qwen2-7b-rag-ko-checkpoint-141'), pr_revision=None, pr_num=None)